In [1]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# Данный файл содержит алгоритмы построения графиков различных зависимостей

In [2]:
from lrnn import (
    LRNN,
    image_to_blocks,
    compress_image,
    decompress_image
)
import numpy as np
from PIL import Image
import plotly.graph_objects as go

In [3]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [4]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('car.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.1))])


network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=2000)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 4.807507403561401
Epoch 1/15000, Loss: 2711.3618453305535
Epoch 2/15000, Loss: 2557.6621925644235
Epoch 3/15000, Loss: 2475.258679943778
Epoch 4/15000, Loss: 2417.632783555434
Epoch 5/15000, Loss: 2380.231238426982
Epoch 6/15000, Loss: 2342.1620264472685
Epoch 7/15000, Loss: 2309.1111813099096
Epoch 8/15000, Loss: 2274.9756073815247
Epoch 9/15000, Loss: 2244.163642700621
Epoch 10/15000, Loss: 2209.9174866011754
Epoch 11/15000, Loss: 2176.762726331918
Epoch 12/15000, Loss: 2151.076245328437
Epoch 13/15000, Loss: 2119.096409560967
Epoch 14/15000, Loss: 2088.551515837818
Epoch 15/15000, Loss: 2066.859954950907
Epoch 16/15000, Loss: 2037.9519749876042
Epoch 17/15000, Loss: 2007.214169138903
Epoch 18/15000, Loss: 1984.5278874534074
Compression coefficient: 2.466780686751743
Z = 4.807507403561401


## Relation between learning rate and epochs 

In [5]:
alphas = np.linspace(0.0005, 0.002, endpoint=True, num=10)
epochs_amount: list[int] = []
for alpha in alphas:
    dl = LRNN(n, p, alpha)
    dl.train(train, 15000, learn_by_loss=True, max_loss=2500)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Коэффициент обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 4236.938222751431
Epoch 2/15000, Loss: 3484.180371045542
Epoch 3/15000, Loss: 3189.260318768548
Epoch 4/15000, Loss: 3011.669497272544
Epoch 5/15000, Loss: 2881.276869820687
Epoch 6/15000, Loss: 2778.5271020314094
Epoch 7/15000, Loss: 2692.2216902817945
Epoch 8/15000, Loss: 2618.0774328619927
Epoch 9/15000, Loss: 2550.909404671559
Epoch 10/15000, Loss: 2492.9485554345274
Epoch 1/15000, Loss: 2839.8414865006275
Epoch 2/15000, Loss: 2693.1360041431985
Epoch 3/15000, Loss: 2607.2513448790714
Epoch 4/15000, Loss: 2552.687511943762
Epoch 5/15000, Loss: 2507.298483254257
Epoch 6/15000, Loss: 2468.925192076157
Epoch 1/15000, Loss: 2583.6308901486123
Epoch 2/15000, Loss: 2473.2919244338796
Epoch 1/15000, Loss: 2711.3618453305535
Epoch 2/15000, Loss: 2557.6621925644235
Epoch 3/15000, Loss: 2475.258679943778
Epoch 1/15000, Loss: 3936.829769485873
Epoch 2/15000, Loss: 3111.737865979727
Epoch 3/15000, Loss: 2782.4748866175037
Epoch 4/15000, Loss: 2574.4154123589738
Epoch 5/150

## Relationship between hidden layer dimension and epochs

In [6]:
p_s = (10, 15, 20, 25, 30, 35, 40)
epochs_amount: list[int] = []
compression_rates: list[float] = []
for p in p_s:    
    dl = LRNN(n, p, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=2000)
    epochs_amount.append(dl.epoch)

    compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
    compression_info_size = (
        compressed.size * compressed.itemsize +
        network.W_dec.size * network.W_dec.itemsize +
        np.array(shape).size * np.array(shape).itemsize +
        np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
    ) * 8
    compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)

fig = go.Figure()
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Коэффициент сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 2711.530340151997
Epoch 2/15000, Loss: 2627.4342824417113
Epoch 3/15000, Loss: 2584.4433081814545
Epoch 4/15000, Loss: 2554.9317094896987
Epoch 5/15000, Loss: 2534.9026828390165
Epoch 6/15000, Loss: 2513.169374394399
Epoch 7/15000, Loss: 2502.1969062652906
Epoch 8/15000, Loss: 2488.498212420498
Epoch 9/15000, Loss: 2473.9177579122024
Epoch 10/15000, Loss: 2462.192764063666
Epoch 11/15000, Loss: 2446.2233861397904
Epoch 12/15000, Loss: 2434.3266106226665
Epoch 13/15000, Loss: 2422.2842801275906
Epoch 14/15000, Loss: 2411.6003404359703
Epoch 15/15000, Loss: 2400.394580388398
Epoch 16/15000, Loss: 2389.2188299392237
Epoch 17/15000, Loss: 2377.8587283887287
Epoch 18/15000, Loss: 2370.415189159642
Epoch 19/15000, Loss: 2355.5866828640064
Epoch 20/15000, Loss: 2349.037073715172
Epoch 21/15000, Loss: 2336.4108758520233
Epoch 22/15000, Loss: 2326.0241551134122
Epoch 23/15000, Loss: 2320.0434526088934
Epoch 24/15000, Loss: 2312.9298107797795
Epoch 25/15000, Loss: 2301.76260

## Relationship between max error and epochs

In [7]:
max_errors = list(range(1000, 3000, 100))
epochs_amount: list[int] = []
for err in max_errors:    
    dl = LRNN(n, 50, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=err)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 4665.672604264205
Epoch 2/15000, Loss: 3167.233233628509
Epoch 3/15000, Loss: 2634.8403086877834
Epoch 4/15000, Loss: 2326.529201195101
Epoch 5/15000, Loss: 2117.6977322785
Epoch 6/15000, Loss: 1966.0699706354687
Epoch 7/15000, Loss: 1850.7562318353914
Epoch 8/15000, Loss: 1752.8642179101196
Epoch 9/15000, Loss: 1675.6166612566767
Epoch 10/15000, Loss: 1605.2086052563445
Epoch 11/15000, Loss: 1545.2871284315154
Epoch 12/15000, Loss: 1502.1897451315572
Epoch 13/15000, Loss: 1457.8382660826855
Epoch 14/15000, Loss: 1418.4960833015307
Epoch 15/15000, Loss: 1381.4662302854656
Epoch 16/15000, Loss: 1351.3975139782337
Epoch 17/15000, Loss: 1321.3222042233278
Epoch 18/15000, Loss: 1295.1961469698322
Epoch 19/15000, Loss: 1271.9741931978847
Epoch 20/15000, Loss: 1252.9975071214956
Epoch 21/15000, Loss: 1231.401643556246
Epoch 22/15000, Loss: 1211.4050165965873
Epoch 23/15000, Loss: 1194.2166375827117
Epoch 24/15000, Loss: 1179.7361139187815
Epoch 25/15000, Loss: 1165.08304

## Relationship between image and epochs

In [8]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 50 

img = Image.open('images/winter.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1500)

print(network.epoch)

# compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg_array = np.asarray(dimg)
# dimg.save('compression-decompression_test.jpg')

Z = 1.9230473377332657
Epoch 1/15000, Loss: 5209.6318296366
Epoch 2/15000, Loss: 3162.699429785375
Epoch 3/15000, Loss: 2710.4352869350473
Epoch 4/15000, Loss: 2450.4228952694975
Epoch 5/15000, Loss: 2269.9157952415144
Epoch 6/15000, Loss: 2136.711794596953
Epoch 7/15000, Loss: 2032.7831097109295
Epoch 8/15000, Loss: 1947.5842874855157
Epoch 9/15000, Loss: 1876.4486331339565
Epoch 10/15000, Loss: 1814.9423286750928
Epoch 11/15000, Loss: 1761.5728006060206
Epoch 12/15000, Loss: 1713.8186292179817
Epoch 13/15000, Loss: 1670.877801951604
Epoch 14/15000, Loss: 1631.9953954299272
Epoch 15/15000, Loss: 1596.7104192915301
Epoch 16/15000, Loss: 1564.6706844158416
Epoch 17/15000, Loss: 1534.7862598245742
Epoch 18/15000, Loss: 1506.9203029658981
Epoch 19/15000, Loss: 1481.1870920688068
19
